To Do!

- Percentage of people who have gotten their first job vs how many looking
- Used resources
    - In which country the resource originated
    - Traffic data for resource?
- Spoken language

In [ ]:
import os.path
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import json
import folium
from collections import defaultdict
from branca.colormap import linear
from IPython.core.display import HTML

In [ ]:
HTML("""
<style>
.output_png img{
    display: block;
    margin-left: auto;
    margin-right: auto;
}
</style>
""")

In [ ]:
def set_plot_font_size(size):
    mpl.rcParams['font.size'] = size

In [ ]:
headers = 'Age,"AttendedBootcamp","BootcampFinish","BootcampFullJobAfter","BootcampLoanYesNo","BootcampMonthsAgo","BootcampName","BootcampPostSalary","BootcampRecommend","ChildrenNumber","CityPopulation","CodeEventBootcamp","CodeEventCoffee","CodeEventConferences","CodeEventDjangoGirls","CodeEventGameJam","CodeEventGirlDev","CodeEventHackathons","CodeEventMeetup","CodeEventNodeSchool","CodeEventNone","CodeEventOther","CodeEventRailsBridge","CodeEventRailsGirls","CodeEventStartUpWknd","CodeEventWomenCode","CodeEventWorkshop","CommuteTime","CountryCitizen","CountryLive","EmploymentField","EmploymentFieldOther","EmploymentStatus","EmploymentStatusOther","ExpectedEarning","FinanciallySupporting","Gender","HasChildren","HasDebt","HasFinancialDependents","HasHighSpdInternet","HasHomeMortgage","HasServedInMilitary","HasStudentDebt","HomeMortgageOwe","HoursLearning","ID.x","ID.y","Income","IsEthnicMinority","IsReceiveDiabilitiesBenefits","IsSoftwareDev","IsUnderEmployed","JobApplyWhen","JobPref","JobRelocateYesNo","JobRoleInterest","JobRoleInterestOther","JobWherePref","LanguageAtHome","MaritalStatus","MoneyForLearning","MonthsProgramming","NetworkID","Part1EndTime","Part1StartTime","Part2EndTime","Part2StartTime","PodcastChangeLog","PodcastCodeNewbie","PodcastCodingBlocks","PodcastDeveloperTea","PodcastDotNetRocks","PodcastHanselminutes","PodcastJSJabber","PodcastJsAir","PodcastNone","PodcastOther","PodcastProgrammingThrowDown","PodcastRubyRogues","PodcastSEDaily","PodcastShopTalk","PodcastTalkPython","PodcastWebAhead","ResourceBlogs","ResourceBooks","ResourceCodeWars","ResourceCodecademy","ResourceCoursera","ResourceDevTips","ResourceEdX","ResourceEggHead","ResourceFCC","ResourceGoogle","ResourceHackerRank","ResourceKhanAcademy","ResourceLynda","ResourceMDN","ResourceOdinProj","ResourceOther","ResourcePluralSight","ResourceReddit","ResourceSkillCrush","ResourceSoloLearn","ResourceStackOverflow","ResourceTreehouse","ResourceUdacity","ResourceUdemy","ResourceW3Schools","ResourceYouTube","SchoolDegree","SchoolMajor","StudentDebtOwe"'
headers = map((lambda (i, header): '{0} {1}'.format(i, header.replace('"', ''))), enumerate(headers.split(',')))

for h in headers:
    print h

In [ ]:
base_dir = os.path.join('data')
input_path = os.path.join('2016-FCC-New-Coders-Survey-Data.csv')

file_name = os.path.join(base_dir, input_path)

# Code to read each character and parse it
# Results in a beautifully split line! :)
def split(l):
    mappedLine = []
    tmp = ''
    in_string = False
    
    for c in l:
        if c == ',':
            if in_string:
                tmp += c
            else:
                mappedLine.append(tmp)
                tmp = ''
        elif c == '"':
            in_string = not in_string
        else:
            tmp += c
    
    return mappedLine

filtered_set = sc.textFile(file_name).filter(lambda l: 'AttendedBootcamp' not in l).map(split)

print 'Count of records : ' + str(filtered_set.count())

In [ ]:
def build_data_gender(line):
    gender = line[36].lower()
    
    if gender == 'male':
        return [1, 0, 0, 0]
    elif gender == 'female':
        return [0, 1, 0, 0]
    elif gender == 'na':
        return [0, 0, 0, 1]
    else:
        return [0, 0, 1, 0]

featured_gender = filtered_set.map(build_data_gender)

In [ ]:
%matplotlib inline

# Make the percentages bigger! (readability)
set_plot_font_size(20.0)
#mpl.rcParams['font.size'] = 20.0

fracs1 = np.mean(featured_gender.collect(), axis=0)

explode = (0.0, 0.3, 0.3, 0.3)
labels = ['Male', 'Female', 'Other', 'Not answered']
colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral']
fig = plt.figure(figsize=(7, 7))
fig.suptitle('Gender distribution', fontsize=14, fontweight='bold')
ax1 = fig.add_subplot(1,1,1)
ax1.pie(fracs1, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=90)

plt.show()

pass

In [ ]:
"""
Age barplot calculations
"""
def build_data_age(line):
    age = line[0]
    label_gender = line[36]
    
    if age == 'NA':
        return (label_gender, [1, 0, 0, 0, 0])
    else:
        age = int(age)
        if age <= 20:
            return (label_gender, [0, 1, 0, 0, 0])
        elif age > 20 and age <= 35:
            return (label_gender, [0, 0, 1, 0, 0])
        elif age > 35 and age <= 50:
            return (label_gender, [0, 0, 0, 1, 0])
        else:
            return (label_gender, [0, 0, 0, 0, 1])

featured_age = filtered_set.map(build_data_age)

males = featured_age.filter(lambda x: x[0] == 'male').map(lambda x: x[1])
females = featured_age.filter(lambda x: x[0] == 'female').map(lambda x: x[1])
not_answered = featured_age.filter(lambda x: x[0] == 'NA').map(lambda x: x[1])

sums_male = np.sum(males.collect(), axis=0)
sums_female = np.sum(females.collect(), axis=0)
sums_not_answered = np.sum(not_answered.collect(), axis=0)

print sums_male
print sums_female
print sums_not_answered

sum_total = np.sum([sums_male, sums_female, sums_not_answered])
print sum_total

In [ ]:
%matplotlib inline

#Gender - Age arrays
males = featured_age.filter(lambda x: x[0] == 'male').map(lambda x: x[1])
females = featured_age.filter(lambda x: x[0] == 'female').map(lambda x: x[1])
not_answered = featured_age.filter(lambda x: x[0] == 'NA').map(lambda x: x[1])

sums_male = np.sum(males.collect(), axis=0)
sums_female = np.sum(females.collect(), axis=0)
sums_not_answered = np.sum(not_answered.collect(), axis=0)

#Age
N = 5
width = 0.5
LABELS = ["NA","<=20","21 - 35","36 - 50","50 <"]

#sum_age = np.sum(featured_age.collect(), axis = 0)

#ind = np.arange(N) = x_axis
x_axis = np.arange(N)

fig = plt.figure(figsize=(15, 7))
fig.suptitle('Age Distribution', fontsize=14, fontweight='bold')
ax = fig.add_subplot(1,1,1)

p1 = ax.bar(x_axis, sums_male, width, color= 'gold', align='center')
p2 = ax.bar(x_axis, sums_female, width, color='lightcoral', bottom=sums_male, align='center')
p3 = ax.bar(x_axis, sums_not_answered, width, color='yellowgreen', bottom= sums_male + sums_female, align='center')

ax.set_xlabel('Age')
ax.set_ylabel('Amount')
plt.xticks(x_axis, LABELS)

pass

In [ ]:
def build_data_employment(line):
    status = line[32]
    
    if status == 'Employed for wages':
        return [1, 0, 0, 0, 0, 0, 0]
    elif status == 'Self-employed freelancer' or status == 'Self-employed business owner':
        return [0, 1, 0, 0, 0, 0, 0]
    elif status == 'Doing an unpaid internship':
        return [0, 0, 1, 0, 0, 0, 0]
    elif status == 'Not working but looking for work':
        return [0, 0, 0, 1, 0, 0, 0]
    elif status == 'Not working and not looking for work':
        return [0, 0, 0, 0, 1, 0, 0]
    elif status == 'NA':
        return [0, 0, 0, 0, 0, 1, 0]
    else:
        return [0, 0, 0, 0, 0, 0, 1]

featured_employ = filtered_set.map(build_data_employment)

In [ ]:
%matplotlib inline

# Make the percentages bigger! (readability)
mpl.rcParams['font.size'] = 20.0

fracs_employ = np.mean(featured_employ.collect(), axis=0)

explode = (0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2)
labels = ['Employed', 'Self-employed', 'Internship','Not working but looking', 'Not working or looking', 'Not answered', 'Other']
colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral', 'lime', 'aqua', 'orange']
fig = plt.figure(figsize=(7, 7))
fig.suptitle('Employment', fontsize=14, fontweight='bold')
ax1 = fig.add_subplot(111)
ax1.pie(fracs_employ, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=True, startangle=90)

plt.show()

pass

In [ ]:
countries_with_continents_path = os.path.join('data', 'countries_with_continent.json')
countries_with_continents = json.load(open(countries_with_continents_path))

continents_with_countries = defaultdict(lambda: [])

for country in countries_with_continents:
    continents_with_countries[countries_with_continents[country].lower().replace(' ', '')].append(country.lower())

In [ ]:
# Indexes for continents
INDEX_ASIA = 0
INDEX_AFRICA = 1
INDEX_EUROPE = 2
INDEX_NORTH_AMERICA = 3
INDEX_OCEANIA = 4
INDEX_SOUTH_AMERICA = 5
INDEX_NA_OR_OTHER = 6

# The exceptions are named differently in the dataset and in the countries dataset
def remove_exceptions(country):
    exceptions = {
        'united states of america': 'united states',
        'korea south': 'korea, south',
        'netherlands (holland, europe)': 'netherlands',
        'bosnia & herzegovina': 'bosnia and herzegovina',
        'trinidad & tobago': 'trinidad and tobago',
        'great britain': 'united kingdom',
        'virgin islands (usa)': 'virgin islands',
        'virgin islands (british)': 'british virgin islands',
        'republic of serbia': 'serbia',
        'myanmar': 'burma (myanmar)',
        'republic of montenegro': 'montenegro',
    }
    
    if country in exceptions:
        return exceptions[country]
    return country

def build_data_continent(line):
    country_live = remove_exceptions(line[29].lower())
    
    if country_live in continents_with_countries['asia']:
        return [1, 0, 0, 0, 0, 0]
    elif country_live in continents_with_countries['africa']:
        return [0, 1, 0, 0, 0, 0]
    elif country_live in continents_with_countries['europe']:
        return [0, 0, 1, 0, 0, 0]
    elif country_live in continents_with_countries['northamerica']:
        return [0, 0, 0, 1, 0, 0]
    elif country_live in continents_with_countries['oceania']:
        return [0, 0, 0, 0, 1, 0]
    elif country_live in continents_with_countries['southamerica']:
        return [0, 0, 0, 0, 0, 1]
    else:
        return None

built_continents = filtered_set.map(build_data_continent).filter(lambda x: x) # Last filter removes all NA and Other elements
people_per_continent = np.sum(built_continents.collect(), axis=0)

continents = {
    'asia': people_per_continent[INDEX_ASIA],
    'africa': people_per_continent[INDEX_AFRICA],
    'europe': people_per_continent[INDEX_EUROPE],
    'northamerica': people_per_continent[INDEX_NORTH_AMERICA],
    'oceania': people_per_continent[INDEX_OCEANIA],
    'southamerica': people_per_continent[INDEX_SOUTH_AMERICA],
}

print people_per_continent

In [ ]:
minimum = min(people_per_continent)
maximum = max(people_per_continent)

colormap = linear.YlGn.scale(
    minimum,
    maximum
)

colormap

In [ ]:
continents_path = os.path.join('data', 'continents.json')
continents_geojson = json.load(open(continents_path))

In [ ]:
m = folium.Map([0, 0], zoom_start=2)

folium.GeoJson(
    continents_geojson,
    style_function=lambda feature: {
        'fillColor': colormap(continents[feature['properties']['continent'].lower()]),
        'fillOpacity' : 0.9,
        'color': 'black',
        'weight': 2,
        'dashArray': '1, 1'
    }
).add_to(m)

marker_cluster = folium.MarkerCluster().add_to(m)
def add_marker(latitude, longitude, text):
    folium.Marker(
        location=[latitude, longitude],
        popup=text,
        icon=None,
    ).add_to(marker_cluster)

add_marker(40.950403, -101.777344, 'North America: {}'.format(people_per_continent[INDEX_NORTH_AMERICA]))
add_marker(-14.30224, -57.832031, 'South America: {}'.format(people_per_continent[INDEX_SOUTH_AMERICA]))
add_marker(10.276486, 22.851563, 'Africa: {}'.format(people_per_continent[INDEX_AFRICA]))
add_marker(50.151876, 14.414063, 'Europe: {}'.format(people_per_continent[INDEX_EUROPE]))
add_marker(53.621476, 92.460938, 'Asia: {}'.format(people_per_continent[INDEX_ASIA]))
add_marker(-23.921549, 130.429688, 'Oceania: {}'.format(people_per_continent[INDEX_OCEANIA]))

# Show the map
m

In [ ]:
INDEX_RESOURCES_START = 84
INDEX_RESOURCES_END = 109
def build_data_resources(line):
    def change_na_to_0(resource):
        """
        # re.match(r'^\d+$', resource) can also be used, but is unnecessary because the only numbers in the resources is '1'
        if resource == '1':
            return int(resource)
        else:
            return 0
        """
        
        if resource == 'NA':
            return 0
        else:
            return 1
    return map(change_na_to_0, line[INDEX_RESOURCES_START : INDEX_RESOURCES_END + 1]) # Take subset of each array, which contain the resources

built_resources = filtered_set.map(build_data_resources)
resources = np.sum(built_resources.collect(), axis=0)

print resources

In [ ]:
"""
Resource list

84 ResourceBlogs - 0
85 ResourceBooks - 1
86 ResourceCodeWars - 2
87 ResourceCodecademy - 3
88 ResourceCoursera - 4
89 ResourceDevTips - 5
90 ResourceEdX - 6
91 ResourceEggHead - 7
92 ResourceFCC - 8 (https://www.freecodecamp.com/)
93 ResourceGoogle - 9
94 ResourceHackerRank - 10
95 ResourceKhanAcademy - 11
96 ResourceLynda - 12
97 ResourceMDN - 13
98 ResourceOdinProj - 14
99 ResourceOther - 15
100 ResourcePluralSight - 16
101 ResourceReddit - 17
102 ResourceSkillCrush - 18
103 ResourceSoloLearn - 19
104 ResourceStackOverflow - 20
105 ResourceTreehouse - 21
106 ResourceUdacity - 22
107 ResourceUdemy - 23
108 ResourceW3Schools - 24
109 ResourceYouTube - 25
"""

set_plot_font_size(10.0)

#Age
N = 5
width = 0.5
LABELS = ['Blogs', 'Books', 'CodeWars', 'Codecademy', 'Coursera', 'DevTips', 'Edx', 'EggHead', 'FCC', 'Google', 'HackerRank',
          'KhanAcademy', 'Lynda', 'MDN', 'OdingProj', 'Other', 'PluralSight', 'Reddit', 'SkillCrush', 'SoloLearn',
          'StackOverflow', 'Treehouse', 'Udacity', 'Udemy', 'W3Schools', 'YouTube']

x_axis = np.arange(len(LABELS))

fig = plt.figure(figsize=(15, 7))
ax = fig.add_subplot(1,1,1)

p1 = ax.bar(x_axis, resources, width, color='gold', align='center')

ax.set_xlabel('Resource')
ax.set_ylabel('Amount')
plt.xticks(x_axis, LABELS, rotation=90)

pass

In [ ]:
INDEX_RESOURCES_START = 11
INDEX_RESOURCES_END = 26
def build_data_resources(line):
    def change_na_to_0(resource):
        """
        # re.match(r'^\d+$', resource) can also be used, but is unnecessary because the only numbers in the resources is '1'
        if resource == '1':
            return int(resource)
        else:
            return 0
        """
        if resource == 'NA':
            return 0
        else:
            return 1
    return map(change_na_to_0, line[INDEX_RESOURCES_START : INDEX_RESOURCES_END + 1]) # Take subset of each array, which contain the resources

built_resources = filtered_set.map(build_data_resources)
resources = np.sum(built_resources.collect(), axis=0)

print resources

In [ ]:
"""
Resource list

11 CodeEventBootcamp
12 CodeEventCoffee
13 CodeEventConferences
14 CodeEventDjangoGirls
15 CodeEventGameJam
16 CodeEventGirlDev
17 CodeEventHackathons
18 CodeEventMeetup
19 CodeEventNodeSchool
20 CodeEventNone
21 CodeEventOther
22 CodeEventRailsBridge
23 CodeEventRailsGirls
24 CodeEventStartUpWknd
25 CodeEventWomenCode
26 CodeEventWorkshop
"""

set_plot_font_size(10.0)

width = 0.5
LABELS = ['CodeEventBootcamp', 'CodeEventCoffee', 'CodeEventConferences', 'CodeEventDjangoGirls', 'CodeEventGameJam', 
          'CodeEventGirlDev', 'CodeEventHackatons', 'CodeEventMeetup', 'CodeEventNodeSchool', 'CodeEventNone', 'CodeEventOther',
          'CodeEventRailsBridge', 'CodeEventRailsGirls', 'CodeEventStartUpWknd', 'CodeEventWomenCode', 'CodeEventWorkshop']

x_axis = np.arange(len(LABELS))

fig = plt.figure(figsize=(15, 7))
ax = fig.add_subplot(1,1,1)

p1 = ax.bar(x_axis, resources, width, color='orange', align='center')

ax.set_xlabel('Resource')
ax.set_ylabel('Amount')
plt.xticks(x_axis, LABELS, rotation=90)

pass